In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import missingno as msno
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', None)

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [42]:
# %reload_kedro

In [43]:
catalog.list()

['razer_raw',
 'razer_preprocessed',
 'parameters',
 'params:variable',
 'params:fixed']

In [11]:
# catalog.load('parameters')
params = catalog.load('parameters')

[06/20/23 11:18:13] INFO     Loading data from 'parameters' (MemoryDataSet)...                  ]8;id=215204;file://C:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=191011;file://C:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

In [7]:
# params.keys()
credit_value = None
for item in params['variable_channels']:
    if 'credit' in item:
        credit_value = item['credit']
        break
        
print(credit_value)

0.0135


In [12]:
data_raw = catalog.load('raw_razer')

[06/20/23 11:18:27] INFO     Loading data from 'raw_razer' (ExcelDataSet)...                    ]8;id=372030;file://C:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=978339;file://C:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

In [14]:
data_raw.head(10)

,Order Id,Txn Id,Amount,Status,Type,Provider,Channel,Payment Date
0,D095M1N4QA,95AYFMS1BB,MYR 100.00,SUCCESS,RAZERPAY,RAZERPAY,FPX_MB2U,"20 Jun 2023, 10:54:15 am"
1,6H7UWB89UK,HMQ97ZU54X,MYR 200.00,SUCCESS,RAZERPAY,RAZERPAY,FPX_AMB,"20 Jun 2023, 10:47:54 am"
2,0C43D647F3,XUDWQ0IKAD,MYR 300.00,SUCCESS,RAZERPAY,RAZERPAY,FPX_PBB,"20 Jun 2023, 10:45:35 am"
3,69D31SUP8C,48QFBGA9BZ,"MYR 1,000.00",SUCCESS,RAZERPAY,RAZERPAY,Credit,"20 Jun 2023, 10:20:41 am"
4,V34R5NJLV3,RBDOIGTZND,MYR 400.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"20 Jun 2023, 9:52:30 am"
5,517XZUHNP9,1BQGSXS13I,MYR 600.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"20 Jun 2023, 9:50:42 am"
6,J7LYSHS26E,PNXOLXGQ9O,MYR 100.00,SUCCESS,RAZERPAY,RAZERPAY,FPX_MB2U,"20 Jun 2023, 9:47:16 am"
7,WUUWKYV24R,TG74I8ZUXZ,MYR 500.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"20 Jun 2023, 9:43:53 am"
8,3ADDB6L4ZZ,B4MEWY4YOR,MYR 100.00,PENDING,RAZERPAY,RAZERPAY,NaN,"20 Jun 2023, 9:31:29 am"
9,KFE70VPUDP,9G9S3BPWMX,MYR 100.00,FAILED,RAZERPAY,RAZERPAY,Credit,"19 Jun 2023, 4:56:21 pm"


In [ ]:
# # Import date class from datetime module
# from datetime import date
 
# # Returns the current local date
# today = date.today()
# print("Today date is: ", today)

In [21]:
data_success = data_raw[data_raw['Status'] == 'SUCCESS']

In [15]:
# Define a function to transform column names
def transform_column_name(col):
    # Convert capital letter to small letter
    col = col.lower()
    # Replace space with underscore
    col = re.sub(' ', '_', col)
    # Remove leading and trailing special characters
    col = re.sub('^[^a-zA-Z0-9]*|[^a-zA-Z0-9]*$', '', col)
    # Replace special characters with underscore
    col = re.sub('[^a-zA-Z0-9]+', '_', col)
    return col

In [22]:
# Apply the function to the DataFrame columns and rename them
new_columns = {old_col: transform_column_name(old_col) for old_col in data_success.columns}
data_success = data_success.rename(columns=new_columns)

In [49]:
data_success.head(10)

,order_id,txn_id,amount,status,type,provider,channel,payment_date,currency,txn_amount,channel_copy,rate_type,rate,txn_charge,net_amount
0,D095M1N4QA,95AYFMS1BB,MYR 100.00,SUCCESS,RAZERPAY,RAZERPAY,FPX_MB2U,2023-06-20,MYR,100.0,fpx_mb2u,fixed,1.0000,1.00,99.00
1,6H7UWB89UK,HMQ97ZU54X,MYR 200.00,SUCCESS,RAZERPAY,RAZERPAY,FPX_AMB,2023-06-20,MYR,200.0,fpx_amb,fixed,1.0000,1.00,199.00
2,0C43D647F3,XUDWQ0IKAD,MYR 300.00,SUCCESS,RAZERPAY,RAZERPAY,FPX_PBB,2023-06-20,MYR,300.0,fpx_pbb,fixed,1.0000,1.00,299.00
3,69D31SUP8C,48QFBGA9BZ,"MYR 1,000.00",SUCCESS,RAZERPAY,RAZERPAY,Credit,2023-06-20,MYR,1000.0,credit,variable,0.0135,13.50,986.50
4,V34R5NJLV3,RBDOIGTZND,MYR 400.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,2023-06-20,MYR,400.0,credit,variable,0.0135,5.40,394.60
5,517XZUHNP9,1BQGSXS13I,MYR 600.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,2023-06-20,MYR,600.0,credit,variable,0.0135,8.10,591.90
6,J7LYSHS26E,PNXOLXGQ9O,MYR 100.00,SUCCESS,RAZERPAY,RAZERPAY,FPX_MB2U,2023-06-20,MYR,100.0,fpx_mb2u,fixed,1.0000,1.00,99.00
7,WUUWKYV24R,TG74I8ZUXZ,MYR 500.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,2023-06-20,MYR,500.0,credit,variable,0.0135,6.75,493.25
10,9S2KCRBJPK,03JXRBF9I2,MYR 300.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,2023-06-19,MYR,300.0,credit,variable,0.0135,4.05,295.95
11,DNJS4LG3YQ,ZLW69VP0E7,MYR 100.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,2023-06-19,MYR,100.0,credit,variable,0.0135,1.35,98.65


In [25]:
data_success.dtypes

order_id                object
txn_id                  object
amount                  object
status                  object
type                    object
provider                object
channel                 object
payment_date    datetime64[ns]
currency                object
txn_amount             float64
dtype: object

In [24]:
# Convert the 'payment_date' column to date type
data_success['payment_date'] = pd.to_datetime(data_success['payment_date'])

# Remove the time portion while maintaining the datetime type
data_success['payment_date'] = data_success['payment_date'].dt.floor('D')

# Split the 'amount' column into two columns
data_success[['currency', 'txn_amount']] = data_success.loc[:, ('amount')].str.split(' ', n=1, expand=True)

# Convert the 'txn_amount' column to float datatype
# replace comma with empty if there is any
data_success['txn_amount'] = data_success['txn_amount'].apply(lambda x: x.replace(',', '')).astype(float)

In [27]:
# Convert 'channel' to lowercase and replace spaces with underscores
data_success['channel_copy'] = data_success['channel'].apply(lambda x: x.lower().replace(' ', '_').replace('-', '_'))

In [29]:
params

{'variable': [{'credit': 0.0135},
  {'boost': 0.013},
  {'wechatpaymy': 0.01},
  {'grabpay': 0.013},
  {'mb2u_qrpay_push': 0.01},
  {'tng_ewallet': 0.013},
  {'shopeepay': 0.013},
  {'rpp_duitnowqr': 0.0095},
  {'fpx': 0.012},
  {'fpx_b2b': 0.012},
  {'atome': 0.05}],
 'fixed': [{'fpx_mb2u': 1},
  {'mb2u': 1},
  {'fpx_cimbclicks': 1},
  {'cimb_clicks': 1},
  {'fpx_rhb': 1},
  {'rhb_onl': 1},
  {'fpx_pbb': 1},
  {'pbebank': 1},
  {'fpx_hlb': 1},
  {'hlb_onl': 1},
  {'fpx_bimb': 1},
  {'fpx_amb': 1},
  {'amb_w2w': 1},
  {'fpx_abmb': 1},
  {'alb_onl': 1},
  {'fpx_abb': 1},
  {'affin_epg': 1},
  {'fpx_bmmb': 1},
  {'fpx_bkrm': 1},
  {'fpx_bsn': 1},
  {'fpx_ocbc': 1},
  {'fpx_uob': 1},
  {'fpx_hsbc': 1},
  {'fpx_scb': 1},
  {'fpx_kfh': 1},
  {'fpx_b2b_amb': 1.4},
  {'fpx_b2b_hlb': 1.4},
  {'fpx_b2b_uob': 1.4},
  {'fpx_b2b_abb': 1.4},
  {'fpx_b2b_hsbc': 1.4},
  {'fpx_m2e': 1.4},
  {'fpx_b2b_cimb': 1.4},
  {'fpx_b2b_bimb': 1.4},
  {'fpx_b2b_rhb': 1.4},
  {'fpx_b2b_pbb': 1.4},
  {'fpx_b2b_kfh'

In [30]:
# Create a dictionary of lists containing the keys
channel_dict = {key: [inner_dict_key for inner_dict in value for inner_dict_key in inner_dict.keys()] for key, value in params.items()}

channel_dict

{'variable': ['credit',
  'boost',
  'wechatpaymy',
  'grabpay',
  'mb2u_qrpay_push',
  'tng_ewallet',
  'shopeepay',
  'rpp_duitnowqr',
  'fpx',
  'fpx_b2b',
  'atome'],
 'fixed': ['fpx_mb2u',
  'mb2u',
  'fpx_cimbclicks',
  'cimb_clicks',
  'fpx_rhb',
  'rhb_onl',
  'fpx_pbb',
  'pbebank',
  'fpx_hlb',
  'hlb_onl',
  'fpx_bimb',
  'fpx_amb',
  'amb_w2w',
  'fpx_abmb',
  'alb_onl',
  'fpx_abb',
  'affin_epg',
  'fpx_bmmb',
  'fpx_bkrm',
  'fpx_bsn',
  'fpx_ocbc',
  'fpx_uob',
  'fpx_hsbc',
  'fpx_scb',
  'fpx_kfh',
  'fpx_b2b_amb',
  'fpx_b2b_hlb',
  'fpx_b2b_uob',
  'fpx_b2b_abb',
  'fpx_b2b_hsbc',
  'fpx_m2e',
  'fpx_b2b_cimb',
  'fpx_b2b_bimb',
  'fpx_b2b_rhb',
  'fpx_b2b_pbb',
  'fpx_b2b_kfh',
  'fpx_b2b_deutsche',
  'fpx_b2b_abmb',
  'fpx_b2b_scb',
  'fpx_b2b_ocbc',
  'fpx_b2b_bmmb',
  'fpx_emandate',
  'fpx_directdebit',
  'fpx_agrobank',
  'fpx_b2b_agrobank',
  'fpx_b2b_abbm',
  'fpx_b2b_citibank',
  'fpx_b2b_bkrm',
  'fpx_b2b_pbbe',
  'fpx_b2b_uobr',
  'fpx_emandate_abb',
  'f

In [31]:
# Create the new 'rate_type' column based on the 'channel' column
data_success['rate_type'] = data_success['channel_copy'].apply(lambda x: next(key for key, value in channel_dict.items() if x in value))

In [37]:
# Create a new column containing the values based on the keys in the 'channel' column
data_success['rate'] = data_success['channel_copy'].apply(lambda x: next(d.get(x) for d in params.values() for d in d if x in d))

In [39]:
# Create the 'txn_charge' column based on the conditions of 'rate_type'
data_success['txn_charge'] = np.where(data_success['rate_type'] == 'fixed', data_success['rate'], 
                                      data_success['txn_amount'] * data_success['rate'])

In [48]:
# Calculate net_amount
data_success['net_amount'] = data_success['txn_amount'] - data_success['txn_charge']

In [50]:
data_success.dtypes

order_id                object
txn_id                  object
amount                  object
status                  object
type                    object
provider                object
channel                 object
payment_date    datetime64[ns]
currency                object
txn_amount             float64
channel_copy            object
rate_type               object
rate                   float64
txn_charge             float64
net_amount             float64
dtype: object

In [51]:
# Select on the relevant columns as the final dataset
final_cols = ['payment_date', 'order_id', 'txn_id', 'status', 'type', 'channel',
              'rate_type', 'rate', 'currency', 'txn_amount', 'txn_charge', 
              'net_amount']
razer_final = data_success[final_cols]

In [52]:
razer_final.head(10)

,payment_date,order_id,txn_id,status,type,channel,rate_type,rate,currency,txn_amount,txn_charge,net_amount
0,2023-06-20,D095M1N4QA,95AYFMS1BB,SUCCESS,RAZERPAY,FPX_MB2U,fixed,1.0000,MYR,100.0,1.00,99.00
1,2023-06-20,6H7UWB89UK,HMQ97ZU54X,SUCCESS,RAZERPAY,FPX_AMB,fixed,1.0000,MYR,200.0,1.00,199.00
2,2023-06-20,0C43D647F3,XUDWQ0IKAD,SUCCESS,RAZERPAY,FPX_PBB,fixed,1.0000,MYR,300.0,1.00,299.00
3,2023-06-20,69D31SUP8C,48QFBGA9BZ,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,1000.0,13.50,986.50
4,2023-06-20,V34R5NJLV3,RBDOIGTZND,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,400.0,5.40,394.60
5,2023-06-20,517XZUHNP9,1BQGSXS13I,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,600.0,8.10,591.90
6,2023-06-20,J7LYSHS26E,PNXOLXGQ9O,SUCCESS,RAZERPAY,FPX_MB2U,fixed,1.0000,MYR,100.0,1.00,99.00
7,2023-06-20,WUUWKYV24R,TG74I8ZUXZ,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,500.0,6.75,493.25
10,2023-06-19,9S2KCRBJPK,03JXRBF9I2,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,300.0,4.05,295.95
11,2023-06-19,DNJS4LG3YQ,ZLW69VP0E7,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,100.0,1.35,98.65


In [44]:
razer_preprocess = catalog.load("razer_preprocessed")

[06/20/23 15:02:58] INFO     Loading data from 'razer_preprocessed' (CSVDataSet)...             ]8;id=709620;file://C:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=744923;file://C:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

In [46]:
razer_preprocess.dtypes

order_id              object
txn_id                object
amount                object
status                object
type                  object
provider              object
channel               object
payment_date          object
payment_date_only     object
currency              object
txn_amount           float64
channel_copy          object
dtype: object

In [47]:
razer_preprocess.head(10)

,order_id,txn_id,amount,status,type,provider,channel,payment_date,payment_date_only,currency,txn_amount,channel_copy
0,D095M1N4QA,95AYFMS1BB,MYR 100.00,SUCCESS,RAZERPAY,RAZERPAY,FPX_MB2U,"20 Jun 2023, 10:54:15 am",2023-06-20,MYR,100.0,fpx_mb2u
1,6H7UWB89UK,HMQ97ZU54X,MYR 200.00,SUCCESS,RAZERPAY,RAZERPAY,FPX_AMB,"20 Jun 2023, 10:47:54 am",2023-06-20,MYR,200.0,fpx_amb
2,0C43D647F3,XUDWQ0IKAD,MYR 300.00,SUCCESS,RAZERPAY,RAZERPAY,FPX_PBB,"20 Jun 2023, 10:45:35 am",2023-06-20,MYR,300.0,fpx_pbb
3,69D31SUP8C,48QFBGA9BZ,"MYR 1,000.00",SUCCESS,RAZERPAY,RAZERPAY,Credit,"20 Jun 2023, 10:20:41 am",2023-06-20,MYR,1000.0,credit
4,V34R5NJLV3,RBDOIGTZND,MYR 400.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"20 Jun 2023, 9:52:30 am",2023-06-20,MYR,400.0,credit
5,517XZUHNP9,1BQGSXS13I,MYR 600.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"20 Jun 2023, 9:50:42 am",2023-06-20,MYR,600.0,credit
6,J7LYSHS26E,PNXOLXGQ9O,MYR 100.00,SUCCESS,RAZERPAY,RAZERPAY,FPX_MB2U,"20 Jun 2023, 9:47:16 am",2023-06-20,MYR,100.0,fpx_mb2u
7,WUUWKYV24R,TG74I8ZUXZ,MYR 500.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"20 Jun 2023, 9:43:53 am",2023-06-20,MYR,500.0,credit
8,9S2KCRBJPK,03JXRBF9I2,MYR 300.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"19 Jun 2023, 4:55:24 pm",2023-06-19,MYR,300.0,credit
9,DNJS4LG3YQ,ZLW69VP0E7,MYR 100.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"19 Jun 2023, 4:51:06 pm",2023-06-19,MYR,100.0,credit


In [ ]:
interest_date = pd.to_datetime('2023-06-16')

In [ ]:
data_success[data_success['Payment Date'] == interest_date]